In [1]:
import openl3

Using TensorFlow backend.


In [2]:
model = openl3.models.load_embedding_model(input_repr="mel128", content_type="env", embedding_size=6144)

In [3]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 48000)          0         
_________________________________________________________________
melspectrogram_1 (Melspectro (None, 128, 199, 1)       4329600   
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 199, 1)       4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 199, 64)      640       
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 199, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 128, 199, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 199, 64)      3692

-----

In [4]:
from keras import Model, Input

In [5]:
inp = model.get_layer('input_1').input

In [6]:
oups = [
    model.get_layer('melspectrogram_1').output,
    model.get_layer('batch_normalization_1').output,
    model.get_layer('conv2d_1').output,
    model.get_layer('batch_normalization_2').output,
    model.get_layer('activation_1').output,
    model.get_layer('conv2d_2').output,
    model.get_layer('batch_normalization_3').output,
    model.get_layer('activation_2').output,
    model.get_layer('conv2d_3').output,
    model.get_layer('batch_normalization_4').output,
    #--------
    model.get_layer('batch_normalization_8').output,
    model.get_layer('activation_7').output,
    model.get_layer('audio_embedding_layer').output,
    model.get_layer('max_pooling2d_4').output,
    model.get_layer('flatten_1').output,
]

In [7]:
model2 = Model(inputs=[inp], outputs=oups)

In [8]:
from glob import glob
file_list = sorted(glob('../../data/train/*.wav') + glob('../../data/validate/*.wav'))

In [9]:
file_list = file_list[:5]

In [10]:
import os
os.path.basename(file_list[0])

'01_000006.wav'

In [11]:
import librosa
audio_np, sr = librosa.load(
        file_list[0],
        sr=48000) # 48000 is the required sr by openl3

In [12]:
import numpy as np

In [13]:
TARGET_SR = 48000
hop_size=0.1
hop_len = int(hop_size * TARGET_SR)
frame_len = TARGET_SR
def _pad_audio(audio, frame_len, hop_len):
    """Pad audio if necessary so that all samples are processed"""
    audio_len = audio.size
    if audio_len < frame_len:
        pad_length = frame_len - audio_len
    else:
        pad_length = int(np.ceil((audio_len - frame_len)/float(hop_len))) * hop_len \
                     - (audio_len - frame_len)

    if pad_length > 0:
        audio = np.pad(audio, (0, pad_length), mode='constant', constant_values=0)

    return audio

audio = _pad_audio(audio_np, frame_len, hop_len)

In [14]:
n_frames = 1 + int((len(audio) - frame_len) / float(hop_len))
x = np.lib.stride_tricks.as_strided(audio, shape=(frame_len, n_frames),
        strides=(audio.itemsize, hop_len * audio.itemsize)).T
# Add a channel dimension
x = x.reshape((x.shape[0], 1, x.shape[-1]))

In [15]:
x.shape

(92, 1, 48000)

In [16]:
ans = model2.predict(x, verbose=True)

92/92 [==============================] - 4s 44ms/step


In [17]:
len(ans)

15

In [18]:
import pickle

In [19]:
with open('test.pkl', 'wb') as f:
    pickle.dump(ans, f)